In [1]:
import pandas as pd
import numpy as np
import pydataset as data
import env
import os

database_url_base = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/tidy_data'

In [2]:
adf = pd.read_sql('SELECT * FROM attendance', database_url_base)
adf


,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
# H to = .5,
# A to = 0,
# T>= 10 = A
# and P to = 1
# then take the average.

In [4]:
adf = adf.melt(id_vars = 'Unnamed: 0')
adf.head(9)

,Unnamed: 0,variable,value
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T


In [5]:
adf = adf.rename(columns = {'Unnamed: 0' : 'student', 'variable': 'date', 'value' : 'grade'})
adf.head()

,student,date,grade
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [6]:
adf['grade'] = adf.grade.map({'A': 0.0, 'H': 0.5, 'T': 0.9, 'P':1.0})

In [7]:
adf.head()

,student,date,grade
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9


In [8]:
adf.groupby(by = 'student').grade.mean()

student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: grade, dtype: float64

In [9]:
#SAME OUTPUT WITH A PIVOT TABLE

adf.pivot_table(index = ['student'], values = ['grade'])

,grade
student,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


In [10]:
cdf = pd.read_sql('SELECT * FROM coffee_levels', database_url_base)
cdf.head(9)

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891


In [11]:
cdf.pivot_table(index = 'hour', columns = 'coffee_carafe' ,values = 'coffee_amount').reset_index()

coffee_carafe,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


In [12]:
rdf = pd.read_sql('SELECT * FROM cake_recipes', database_url_base)

In [13]:
rdf.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


In [14]:
rdf[['recipe', 'position']] = rdf['recipe:position'].str.split(':' , expand = True)
rdf.head()

,recipe:position,225,250,275,300,recipe,position
0,a:bottom,61.738655,53.912627,74.414730,98.786784,a,bottom
1,a:top,51.709751,52.009735,68.576858,50.228470,a,top
2,b:bottom,57.095320,61.904369,61.196980,99.248541,b,bottom
3,b:top,82.455004,95.224151,98.594881,58.169349,b,top
4,c:bottom,96.470207,52.001358,92.893227,65.473084,c,bottom


In [15]:
rdf = rdf.drop(columns = 'recipe:position')

In [16]:
rdf_melt = rdf.melt(id_vars = ['recipe', 'position'], var_name = 'temp', value_name = 'score')

In [17]:
rdf_melt.head()

,recipe,position,temp,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


In [18]:
rdf_melt.groupby(by = 'recipe').mean()

,score
recipe,
a,63.922201
b,76.736074
c,75.874748
d,62.864844


In [19]:
rdf_melt.groupby('temp').mean()

,score
temp,
225,71.306022
250,66.577437
275,74.886754
300,66.627655


In [20]:
rdf_melt.sort_values('score', ascending = False).head(1)

,recipe,position,temp,score
26,b,bottom,300,99.248541
